<a href="https://colab.research.google.com/github/MuayThaiLegz/PracticeCrazy/blob/main/ModelTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xformers
!pip install transformers
!pip install torch
!pip install --upgrade accelerate
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch

from sklearn.model_selection import train_test_split

In [ ]:
from nltk.corpus import wordnet as wn
import nltk

nltk.download('wordnet')

def synonym_replace(word):
    synsets = wn.synsets(word)
    if synsets:
        synonyms = set([lemma.name() for syn in synsets for lemma in syn.lemmas()])
        synonyms.discard(word)
        if synonyms:
            return random.choice(list(synonyms))
    return word

def augment_text(text):
    words = text.split()
    for i in range(len(words)):
        if random.random() < 0.05:  # 5% chance to replace a word with a synonym
            words[i] = synonym_replace(words[i])
    return " ".join(words)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
emails

,Matter_Number,Matter_Name,Client_Name,Date_Opened,Docket_Number,Court_Name,Matter_Type,Matter_Subtype,Plaintiff_Org_Size,Defendant_Org_Size,Demand_Amount,Fees_Billed,Outcome_Date,Outcome_Type,Outcome_Amount
0,00003-000047,"Adler, Inc. v. Holmes & Watson",Holmes & Watson,12/30/2013,1:18-cv-01437,USDC S.D. Cal.,Commercial Litigation,Breach of Contract,8000,20000,"$95,409.53","$18,983.38",7/15/2014,Settled,"$88,868.58"
1,00004-000044,"Louis Rocher v. Madam Roussad's, Inc.","Madam Roussad's, Inc.",5/24/2013,1:18-cv-01456,USDC E.D. Va.,Commercial Litigation,Breach of Contract,40,20000,"$90,837.64","$15,326.46",2/18/2014,Verdict - Plaintiff,"$57,502.58"
2,00006-000055,Aragorn Strider v. Horizon Co.,Horizon Co.,1/14/2014,1:18-cv-01743,USDC D. Mass.,Commercial Litigation,Breach of Contract,75000,20000,"$51,387.95","$2,138.15",6/17/2014,Summary Judgment,"$33,832.77"
3,00002-000043,Life Indigo v. Sperizon,Sperizon,3/13/2014,1:18-cv-02018,USDC E.D. Va.,Commercial Litigation,Breach of Contract,3900,20000,"$78,030.95","$11,214.97",11/14/2014,Settled,"$49,818.57"
4,00003-000050,"Peter Watson v. Modak, Inc.","Modak, Inc.",4/1/2014,1:18-cv-02067,USDC E.D. Mich.,Commercial Litigation,Breach of Contract,7500,20000,"$88,571.35","$9,710.03",2/10/2015,Settled,"$56,142.81"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,00009-000001,"Owen Glendower v. Red Keep, Inc.","Red Keep, Inc.",9/29/2016,5:18-cv-05671,USDC W.D. Tex.,Commercial Litigation,Stockholder Suit,880,20000,"$111,581.70","$1,858.65",11/7/2016,Settled,"$69,949.02"
496,00001-000002,"Seandi, Inc. v. Keefe & Cote",Keefe & Cote,5/25/2016,5:18-cv-05678,USDC S.D. Cal.,Commercial Litigation,Stockholder Suit,3700,20000,"$103,007.20","$17,828.81",6/30/2016,Settled,"$95,706.48"
497,00004-000001,"Evan Leigh v. Gracie, Co.","Gracie, Co.",6/20/2016,5:18-cv-05789,USDC N.D. Cal.,Commercial Litigation,Stockholder Suit,700,10000,"$93,480.75","$4,617.57",2/10/2017,Settled,"$85,632.68"
498,00002-000001,"Caliban, Inc. v. McCoy, Inc.","McCoy, Inc.",7/5/2017,5:18-cv-05841,USDC W.D. Tex.,Commercial Litigation,Stockholder Suit,200,20000,"$68,883.35","$4,785.75",8/1/2018,Settled,"$44,330.01"


In [ ]:

emails = pd.read_csv("/content/drive/MyDrive/machineLearning_eduData/ILTACON Data - internal_cases (3).csv")
home_email = emails.astype(str)
email_texts = home_email.apply(lambda row: str(row['Matter_Name']) + " " + str(row['Matter_Subtype']), axis=1).tolist()


In [ ]:
# Save the model, tokenizer, and label encoder
from transformers import BertTokenizer, BertForSequenceClassification
import pickle
from torch.utils.data import DataLoader


# import pickle
# model_path = "./bert_base_Puncased"
# trainer.save_model(model_path)
# tokenizer.save_pretrained(model_path)
# with open(model_path + "/label_encoder.pkl", "wb") as f:
#     pickle.dump(label_encoder, f)


emails = pd.read_csv("/content/drive/MyDrive/machineLearning_eduData/ILTACON Data - internal_cases (3).csv")
home_email = emails.astype(str)

# Load the model, tokenizer, and label encoder
model_path = "//content/drive/MyDrive/machineLearning_eduData/emailSortUnobertbaseuncased"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
with open(model_path + "/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)


def predict(emails, model, tokenizer, label_encoder):
    # Tokenize the new emails
    inputs = tokenizer(emails, truncation=True, padding=True, max_length=512, return_tensors="pt")

    # Create a DataLoader for the new emails
    dataset = torch.utils.data.TensorDataset(inputs['input_ids'], inputs['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=16)

    # Make predictions
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = batch
            outputs = model(**inputs)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).tolist())

    # Decode predictions to label names
    predicted_labels = label_encoder.inverse_transform(predictions)

    return predicted_labels


# Make predictions
predicted_labels = predict(email_texts, model, tokenizer, label_encoder)

# Print results
for email, label in zip(home_email, predicted_labels):
    print(f"Email: {email[:50]}... -> Predicted Label: {label}")



Email: Matter_Number... -> Predicted Label: DEF case
Email: Matter_Name... -> Predicted Label: DEF case
Email: Client_Name... -> Predicted Label: Ape v. Monster case
Email: Date_Opened... -> Predicted Label: Ape v. Monster case
Email: Docket_Number... -> Predicted Label: DEF case
Email: Court_Name... -> Predicted Label: Ape v. Monster case
Email: Matter_Type... -> Predicted Label: Ape v. Monster case
Email: Matter_Subtype... -> Predicted Label: Zebra v. Dog case
Email: Plaintiff_Org_Size... -> Predicted Label: DEF case
Email: Defendant_Org_Size... -> Predicted Label: Ape v. Monster case
Email: Demand_Amount... -> Predicted Label: Ape v. Monster case
Email: Fees_Billed... -> Predicted Label: DEF case
Email: Outcome_Date... -> Predicted Label: Ape v. Monster case
Email: Outcome_Type... -> Predicted Label: Zebra v. Dog case
Email: Outcome_Amount... -> Predicted Label: Ape v. Monster case
